# Correlation Network

Correlation network (graph) desribes the strength of correlation among qubit pairs in a system and shows how qubits in a system is correlated. 

In the network:

Nodes: represent the *qubits*.

Edge: represent *how much* the nodes (qubits) are *correlated* by its weight.

The strength can be any value that quantify how a qubit pair is correlated. In previous studies, quantum Mutual Information (MI) is used. Though it remains unknown, we conjecture that Entanglement of Formation (EoF) and various entanglement witnesses are also useful quantities in certain scenario. 

In quantum computing, people can calculate the correlation network of a objective wavefunction, such as the network of the ground state of a Hamiltonian in VQE, in a approximated way, before carrying out quantum computing on the system. 

By this pre-calculation, people can know that

- Which qubits are more active and which qubits are nearly stationary and can be removed.
- The groups of qubits in which the qubits are inter-entangled, so that the computation can be divided into parts.
- Which part of the system is more active and needs higher precision of operation. 
    - So that physics qubits with higher quality can be assigned to it.

We believe that the potential applications shown above make pre-calculation of correlation network very important for near-term quantum computing because

- The number of gates and qubits allowed on near-term quantum devices is very limited.
- The quality of qubits in near-term is not uniform and full-connectivity qubits set will be small.

To address these problems, we provide modules for

- Easily carrying out correlation network approximation from qubit Hamiltonians.
- Finding the optimal mapping to physical qubits based on the correlation network.
- Finding the subsets of qubits which are most correlated.
- Finding the community (correlated group) structure of a correlation network.

## Classical pre-calculation

### Dependency

The classcial pre-calculation module in Mizore is realized by [dmrgpy](https://github.com/joselado/dmrgpy) (iTensor inside). The users must install dmrgpy before using this module. We remind the users that dmrgpy is based on iTensor and `lapack` is needed before installing. The ubuntu users can use `sudo apt-get install libblas-dev liblapack-dev` to get `lapack` ready.

### Basic Usage

The classical pre-calculation can be easily carried out as following.

In [17]:
from CorrelationNetwork.MostCorrelation._ga_selector import GACorrelationQsubsetSelector
from CorrelationNetwork._quantum_chips import *

selector = GACorrelationQsubsetSelector(IBM_5Q_Yorktown())

# Run MostCorrelation selector with time budget 10 seconds
selector.run(time_budget=10)

# Get Result
results = selector.get_result()

# Get Fitness tuple
results = [(item.fitness, [gene for gene in item.genes]) for item in results]
print(results)


[(0.5, [2, 3, 4]), (0.5, [1, 2, 0]), (0.3333333333333333, [1, 2, 0, 4]), (0.3333333333333333, [0, 1, 2, 3]), (0.3333333333333333, [2, 3, 4, 1]), (0.3333333333333333, [4, 2, 3, 0]), (0.3, [2, 1, 3, 4, 0]), (0.16666666666666666, [4, 0, 1, 3])]


Because `calc_2DM` is made `True`, reduced two-qubit density matrices of the ground state is also calculated. Here we show how to use them calculated the correlation quantities.

In [15]:
from Utilities.WaveLocalProperties import get_mutual_information_by_2DMs,get_EoF_by_2DMs
 
classical_res["MI"]=get_mutual_information_by_2DMs(classical_res["2DM"])
classical_res["EoF"]=get_EoF_by_2DMs(classical_res["2DM"])

print("Mutual Information",classical_res["MI"].tolist())
print("Entanglement of Formation",classical_res["EoF"].tolist())

ModuleNotFoundError: No module named 'openfermion'

We can convert the weight matrix to a network and visulize it as following.

In [12]:
from CorrelationNetwork import get_nx_graph_by_adjacent_mat, draw_graph

G = get_nx_graph_by_adjacent_mat(classical_res["MI"])
draw_graph(G)

NameError: name 'classical_res' is not defined

## Most correlated subsets

In [ ]:
corr_adjacent_mat=[[0.0, 0.05686422974791204, 0.01688342559421084, 0.00923050347926574, 0.0438784337237768, 0.043615348773295606], [0.05686422974791204, 0.0, 0.00923050347926574, 0.016984823342198085, 0.043615348773295606, 0.043794613240691074], [0.01688342559421084, 0.00923050347926574, 0.0, 0.0021556255819239617, 0.0012450419319145478, 0.006317482790596407], [0.00923050347926574, 0.016984823342198085, 0.0021556255819239617, 0.0, 0.006317482790596407, 0.0014123307378750144], [0.0438784337237768, 0.043615348773295606, 0.0012450419319145478, 0.006317482790596407, 0.0, 0.03286268696855375], [0.043615348773295606, 0.043794613240691074, 0.006317482790596407, 0.0014123307378750144, 0.03286268696855375, 0.0]]

In [10]:
from CorrelationNetwork.MostCorrelation._ga_selector import GACorrelationQsubsetSelector
from CorrelationNetwork._quantum_chips import *

selector = GACorrelationQsubsetSelector(IBM_5Q_Yorktown())

# Run MostCorrelation selector with time budget 10 seconds
selector.run(time_budget=10)

# Get Result
results = selector.get_result()

# Get Fitness tuple
results = [(item.fitness, [gene for gene in item.genes]) for item in results]
print(results)



[<CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12cdda898>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12d3134e0>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12ca4fe80>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12cbc4668>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12cbc4d68>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12cdda630>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12ca73630>, <CorrelationNetwork.MostCorrelation._chromosome.Chromosome object at 0x12cbd4198>]


## Community detection

In [11]:
from CorrelationNetwork._community_detection import detect_nx_graph_community, draw_community_graph
from CorrelationNetwork import get_nx_graph_by_adjacent_mat

# Initial graph
G_corr = get_nx_graph_by_adjacent_mat(corr_adjacent_mat)

# Detect community
community_map = detect_nx_graph_community(G_corr)
print(community_map)

# Draw community graph
draw_community_graph(G_corr, community_map, "output")

NameError: name 'corr_adjacent_mat' is not defined

## Optimal Mapping

### Dependency

The optimal mapping module in Mizore is realized by [minorminer](https://github.com/dwavesystems/minorminer).  The ubuntu users can use `pip3 install minorminer` to get `minorminer` ready.

### Basic Usage

Optimal Mapping (graph) desribes the graph embedding of source graph in target graph.

In the network:

Nodes: represent the *qubits*.

Edge: represent *mutual information*.

We feed the generic constructor with both source weighted graph and target weighted graph. The optimal results will be evolved and optimized though a heuristic and generic hybrid algorithm. (Fitness will be evaluated by: $fitness(source\_graph, embedding\_graph) = \sum weight(edge_{source\_graph}) * weight(edge_{embeding\_graph})$ )

In [4]:
from CorrelationNetwork._quantum_chips import *
from CorrelationNetwork._utilities import find_paths

# Initial quantum chip
chip = Rigetti_8Q_Agave()

# Find paths
paths = find_paths(chip)
print(paths)


ImportError: cannot import name 'find_paths' from 'Network._utilities' (/Users/liuyi/quantum/code/Mizore/src/Network/_utilities.py)

In [16]:
from CorrelationNetwork import GAGraphEmbeddingConstructor
from CorrelationNetwork._quantum_chips import *

# Initial quantum chips
rigetti_16Q_Aspen = Rigetti_16Q_Aspen()
ibm_20Q_Johannesburg = IBM_20Q_Johannesburg()

# Search optimal mapping
embeding_selector = GAGraphEmbeddingConstructor(rigetti_16Q_Aspen, ibm_20Q_Johannesburg)
embeding_selector.run(time_budget=5)

# Show optimal mapping
results = embeding_selector.get_result()
best_result = results[0]
print(f'fitness :{best_result.fitness}, mapping:{[gene for gene in best_result.genes]}')

[(0.5, [3, 4, 2]), (0.5, [2, 0, 1]), (0.3333333333333333, [1, 2, 3, 0]), (0.3333333333333333, [0, 1, 4, 2]), (0.3333333333333333, [3, 2, 4, 1]), (0.3333333333333333, [4, 2, 3, 0]), (0.3, [3, 4, 1, 2, 0]), (0.16666666666666666, [3, 4, 1, 0])]
